Price and Carat

In [2]:
library(tidyverse)
library(tidymodels)
library(kknn)
library(recipes)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

In [ ]:
set.seed(5)

diamonds <- read_csv("diamonds.csv")
diamonds_split <- initial_split(diamonds, prop = 0.75, strata = price)
diamonds_train <- training(diamonds_split)
diamonds_test <- testing(diamonds_split)

diam_recipe <- recipe(price ~ carat, data = diamonds_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

diam_spec <- nearest_neighbor(weight_func = "rectangular", 
                              neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("regression")

diam_vfold <- vfold_cv(diamonds_train, v = 5, strata = price)

diam_wkflw <- workflow() |>
  add_recipe(diam_recipe) |>
  add_model(diam_spec)

gridvals <- tibble(neighbors = seq(from = 1, to = 200, by = 3))

diam_results <- diam_wkflw |>
  tune_grid(resamples = diam_vfold, grid = gridvals) |>
  collect_metrics() |>
  filter(.metric == "rmse")

diam_min <- diam_results |>
  filter(mean == min(mean))

kmin <- diam_min |> pull(neighbors)

diam_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = kmin) |>
  set_engine("kknn") |>
  set_mode("regression")

diam_fit <- workflow() |>
  add_recipe(diam_recipe) |>
  add_model(diam_spec) |>
  fit(data = diamonds_train)

diam_summary <- diam_fit |>
  predict(diamonds_test) |>
  bind_cols(diamonds_test) |>
  metrics(truth = price, estimate = .pred) |>
  filter(.metric == 'rmse')

carat_prediction_grid <- tibble(
    carat = seq(
        from = diamonds |> select(carat) |> min(),
        to = diamonds |> select(carat) |> max(),
        by = 10))

diam_preds <- diam_fit |>
  predict(carat_prediction_grid) |>
  bind_cols(carat_prediction_grid)

plot_final <- ggplot(diamonds, aes(x = carat, y = price)) +
  geom_point(alpha = 0.4) +
  geom_line(data = diam_preds, 
            mapping = aes(x = carat, y = .pred), 
            color = "blue") +
  xlab("Carat") +
  ylab("Price (USD)") +
  scale_y_continuous(labels = dollar_format()) +
  ggtitle(paste0("K = ", kmin)) + 
  theme(text = element_text(size = 12))
plot_final

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

Price and Carat + Clarity